In [3]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import numpy as np
import pandas as pd
from pandas.io.parsers import read_csv
from CRS_jour import *
from BOAmodel_JMLRv2 import *
from collections import defaultdict
import pickle
from random import sample,random
import copy
from pylab import *
from random import sample
from sklearn.metrics import auc
%matplotlib inline
rcParams['figure.figsize'] = (10, 10)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
def code_categorical(df,collist):
    for col in collist:
        for value in df[col].unique():
            df[col+'_'+str(value)] = (df[col] == value).astype(int)
    df.drop(collist,axis = 1, inplace = True)

def code_continuous(df,collist,Nlevel):
    for col in collist:
        for q in xrange(1,Nlevel,1):
            threshold = df[col].quantile(float(q)/Nlevel)
            df[col+'_geq_'+str(int(q))+'q'] = (df[col] >= threshold).astype(float)
    df.drop(collist,axis = 1, inplace = True)

def normalize_numeric(df,continuous):
    for col in continuous:
        df[col] = np.true_divide(df[col],max(df[col]))

# Generating Simulated Datasets

In [5]:
Nchain = 1
Niteration = 500
Nrules = 25000
supp = 5
maxlen = 3 # maximum length of rules
Ntrue = 5
Nlevel = 5

for N,J in zip([10000],[20]):
    folder = 'N'+str(int(N/1000))+'k_J'+str(J)+'L3_Nrule'+str(int(Nrules/1000))+'k_Nlevel'+str(Nlevel)+'_Ntrue'+str(Ntrue)+'_confound' #noTS means no trueset attached
    for iteration in range(100):
        print([N,J,folder,iteration])
        experiment = {}
        print('================== iter = {}, Ntrue = {} =================='.format(iteration,Ntrue))
        df = pd.DataFrame(30*np.random.rand(N,J),columns = ['att'+str(i+1) for i in range(J)])  
        df0 = df.copy(deep = True)
        df00 = df.copy(deep = True)
        binary_code(df,df.columns,Nlevel)
        train_index = sample(range(N),int(0.7*N))
        test_index = list(set(range(N)) - set(train_index))
        T = np.random.choice([0, 1], size=(len(train_index),), p=[0.5, 0.5])

#=== true set sugroup ====
        # generate true rule set
        flag = True
        while flag:
            w0 = np.random.rand(J) - 0.5
            g0 = np.dot(df0,w0)
            Y0 = (1.0/(1+np.exp(-g0))>=0.5).astype(int)
            if mean(Y0)>=0.3 and mean(Y0)<=0.7:
                flag = False

        dfn = 1-df.iloc[train_index]
        dfn.columns = [name.strip() + 'neg' for name in df.columns]
        df_train = pd.concat([df.iloc[train_index],dfn],axis = 1)
        Z = np.zeros(len(train_index))
        for rule in TrueSet:
            Z += (np.sum(df_train[rule],axis=1)==len(rule)).astype(int)
        Z = np.array((Z>0).astype(int))

# === inserting random noise ===
#         index = sample(range(len(train_index)),int(0.1*len(train_index)))[0]
#         Z[index] = 1-Z[index]
# #=== linear sugroup ====
#         flag = True
#         while flag:
#             ws = np.random.rand(J) - 0.5
#             gs = np.dot(df0,ws)
#             Z = (1.0/(1+np.exp(-gs))>=0.5).astype(int)
#     #         print 'Mean of Y0 is {}'.format(mean(Y0))
#             if mean(Z)>=0.2 and mean(Z)<=0.6:
#                 flag = False
#=== nonelinear sugroup ====
#         flag = True
#         while flag:
#             for k in range(10):
#                 if random()>0.5:
#                     col = sample(df00.columns.tolist(),1)
#                     df00['new'+str(k+1)] = pow(df00[col],sample([2,3],1)[0])
#                 else:
#                     cols = sample(df00.columns.tolist(),2)
#                     df00['new'+str(k+1)] = df00[cols[0]]*df00[cols[1]]
        
#             ws = np.random.rand(df00.shape[1]) - 0.5
#             gs = np.dot(df00,ws)
#             Z = (1.0/(1+np.exp(-gs))>=0.7).astype(int)
#     #         print 'Mean of Y0 is {}'.format(mean(Y0))
#             if mean(Z)>=0.1 and mean(Z)<=0.5:
#                 flag = False
## ==================================================
                
        ind = np.where(np.multiply(T,Z)==1)
        Y = Y0[train_index][:]
        Y[ind]=1

        dfn = 1-df.iloc[test_index]
        dfn.columns = [name.strip() + 'neg' for name in df.columns]
        df_test = pd.concat([df.iloc[test_index],dfn],axis = 1)

        Z_true = np.zeros(len(test_index))

        for rule in TrueSet:
            Z_true += (np.sum(df_test[rule],axis=1)==len(rule)).astype(int)
        Z_true = (Z_true>0).astype(int)



        experiment['Z_true'] = Z_true #Z[test_index] # 
        experiment['df'] = df_test #testing data
        experiment['test_index'] = test_index
        experiment['Y0'] = Y0
        experiment['T'] = T
        experiment['run_time'] = run_time
        pickle.dump(experiment,open('simulations/'+folder+'_'+str(iteration)+'.pkl','wb'))


[10000, 20, 'N10k_J20L3_Nrule25k_Nlevel5_Ntrue5_confound', 0]
================== iter = 0, Ntrue = 5 ==================
Using random forest to generate rules ...
	Took 0.359s to generate 65 rules
	Took 0.009s to compute patternspace
[['att13_geq_2q', 'att18_geq_2q', 'att5_geq_2q'], ['att3_geq_2qneg'], ['att13_geq_3q', 'att17_geq_2qneg', 'att6_geq_2qneg'], ['att16_geq_2q', 'att18_geq_3q'], ['att18_geq_2qneg', 'att6_geq_2qneg', 'att8_geq_1q']]
Using random forest to generate rules ...
	Took 8.479s to generate 13004 rules
Using average treatment effect to screen 13004 rules
njobs is 5
Evaluating rules 0 to 2600
Evaluating rules 2600 to 5200
Evaluating rules 5200 to 7800
Evaluating rules 7800 to 10400
Evaluating rules 10400 to 13000
Evaluating rules 13000 to 13004
length of all rules = 13004
added rules indices are []
	Took 24.169s to screen 12021 rules
	Took 0.013s to compute patternspace
** chain = 0, max at iter = 0 ** 
 obj = -12259.98843326187,rules = [ 586 1463]
vt:-12259.99 = prior_

C:\Users\wangt\Dropbox\Research\Projects\crs\python code\CRS_jour.py:234: RuntimeWarning: overflow encountered in exp
  if random() <= np.exp(np.true_divide(pt_new -pt_curr,T)):


** chain = 0, max at iter = 7 ** 
 obj = -1989.492817211372,rules = [  38  580 1461 2430 7779]
vt:-1989.49 = prior_rules(-58.33) + prior_w(-265.95) + llh(-1665.21), coef = 1.963667186278125,4.78456423886016 and -2.820897052581969, Zerror = 2239
[['att13_geq_2qneg', 'att4_geq_3q', 'att6_geq_3qneg'], ['att18_geq_2qneg', 'att6_geq_2qneg', 'att8_geq_2q'], ['att3_geq_2qneg'], ['att16_geq_2q', 'att18_geq_1q', 'att18_geq_3q'], ['att13_geq_2q', 'att18_geq_3q', 'att5_geq_2q']]
[10000, 20, 'N10k_J20L3_Nrule25k_Nlevel5_Ntrue5_confound', 1]
================== iter = 1, Ntrue = 5 ==================
Using random forest to generate rules ...
	Took 0.340s to generate 69 rules
	Took 0.008s to compute patternspace
[['att12_geq_2qneg', 'att17_geq_2q', 'att8_geq_3qneg'], ['att12_geq_3q'], ['att16_geq_3q', 'att8_geq_2q'], ['att15_geq_2qneg', 'att16_geq_2qneg'], ['att11_geq_2q', 'att12_geq_3q']]
Using random forest to generate rules ...
	Took 8.070s to generate 13966 rules
Using average treatment effect to 

C:\Users\wangt\Dropbox\Research\Projects\crs\python code\CRS_jour.py:234: RuntimeWarning: overflow encountered in exp
  if random() <= np.exp(np.true_divide(pt_new -pt_curr,T)):


** chain = 0, max at iter = 1 ** 
 obj = -16533.96951426523,rules = [2936 7173]
vt:-16533.97 = prior_rules(-21.89) + prior_w(-0.0) + llh(-16512.07), coef = 0.0,20.0 and -20.0, Zerror = 3621
** chain = 0, max at iter = 3 ** 
 obj = -4258.3398903453635,rules = [7173]
vt:-4258.34 = prior_rules(-16.13) + prior_w(-0.43) + llh(-4241.79), coef = 0.023853911523825728,19.97620274189496 and -19.952348830371132, Zerror = 5048
** chain = 0, max at iter = 143 ** 
 obj = -3569.016810722226,rules = [10]
vt:-3569.02 = prior_rules(-12.2) + prior_w(-0.44) + llh(-3556.37), coef = 0.09965312259827025,20.00032500286721 and -19.90067188026894, Zerror = 4164
[['att16_geq_3q', 'att8_geq_3q']]
[10000, 20, 'N10k_J20L3_Nrule25k_Nlevel5_Ntrue5_confound', 2]
================== iter = 2, Ntrue = 5 ==================
Using random forest to generate rules ...
	Took 0.344s to generate 70 rules
	Took 0.009s to compute patternspace
[['att13_geq_3qneg', 'att2_geq_3q', 'att7_geq_3q'], ['att13_geq_1q', 'att20_geq_2qneg', '

...

C:\Users\wangt\Dropbox\Research\Projects\crs\python code\CRS_jour.py:234: RuntimeWarning: overflow encountered in exp
  if random() <= np.exp(np.true_divide(pt_new -pt_curr,T)):


** chain = 0, max at iter = 1 ** 
 obj = -24927.803635440938,rules = [ 1116  1493  5335  6279  9874 10261]
vt:-24927.8 = prior_rules(-70.99) + prior_w(-0.01) + llh(-24856.8), coef = 0.004747584555888282,19.99584990949601 and -19.991102324940123, Zerror = 2366
** chain = 0, max at iter = 2 ** 
 obj = -20684.47535554178,rules = [ 1116  1493  5335  6279 10261]
vt:-20684.48 = prior_rules(-58.33) + prior_w(-0.02) + llh(-20626.13), coef = 0.0061932117496695015,19.995761787044223 and -19.989568575294552, Zerror = 2557
** chain = 0, max at iter = 3 ** 
 obj = -4762.436745781407,rules = [ 1116  1493  5023  5335  6279 10261]
vt:-4762.44 = prior_rules(-63.4) + prior_w(-140.22) + llh(-4558.82), coef = 0.2721937362806326,8.56715915514021 and -8.294965418859642, Zerror = 2121
** chain = 0, max at iter = 22 ** 
 obj = -3359.7561499737812,rules = [12]
vt:-3359.76 = prior_rules(-12.2) + prior_w(-4.34) + llh(-3343.21), coef = 0.8622002227656513,20.002901822088454 and -19.140701599322806, Zerror = 3722
*

C:\Users\wangt\Dropbox\Research\Projects\crs\python code\CRS_jour.py:234: RuntimeWarning: overflow encountered in exp
  if random() <= np.exp(np.true_divide(pt_new -pt_curr,T)):


** chain = 0, max at iter = 3 ** 
 obj = -10993.028520957158,rules = [   41   996  2389  5311 11988]
vt:-10993.03 = prior_rules(-58.07) + prior_w(-0.02) + llh(-10934.94), coef = 0.0037215388364998073,19.996877140899258 and -19.99315560206276, Zerror = 3128
** chain = 0, max at iter = 10 ** 
 obj = -10933.677722907374,rules = [    9   184  5311  7475 10068]
vt:-10933.68 = prior_rules(-65.22) + prior_w(-0.03) + llh(-10868.43), coef = 0.007272919416208853,19.996753736190776 and -19.989480816774567, Zerror = 2883
** chain = 0, max at iter = 12 ** 
 obj = -10795.810675143965,rules = [    9   184   504  7475 10068]
vt:-10795.81 = prior_rules(-65.22) + prior_w(-0.03) + llh(-10730.56), coef = 0.009481167806433315,19.998417581317078 and -19.988936413510647, Zerror = 2744
** chain = 0, max at iter = 26 ** 
 obj = -10650.65358531208,rules = [    9   133   177   608  5454 10357]
vt:-10650.65 = prior_rules(-79.8) + prior_w(-0.05) + llh(-10570.81), coef = 0.014019835491615725,19.99803809286015 and -

KeyboardInterrupt: 